# Data Augmentation

In [1]:
from keras.preprocessing.image import ImageDataGenerator

# Create image augmenter
dataAugment = ImageDataGenerator(rescale=1.0/255,
                                 rotation_range=40,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 zoom_range=0.2,
                                 shear_range=0.2,
                                 fill_mode='reflect',
                                 horizontal_flip=True,
                                 validation_split=0.2,
                                 data_format='channels_last'
                                )
        
trainDataGenerator = dataAugment.flow_from_directory(directory='Data/Train',
                                                     target_size=(224, 224), # change this to the input size used for your model
                                                     class_mode='categorical',
                                                     subset='training',
                                                     shuffle=True
                                                     # batch_size=
                                                    )

validationDataGenerator = dataAugment.flow_from_directory(directory='Data/Train',
                                                          target_size=(224,224), # change this to the input size used for your model
                                                          class_mode='categorical',
                                                          subset='validation',
                                                          shuffle=True
                                                          # batch_size=
                                                         )

Using TensorFlow backend.


Found 221 images belonging to 3 classes.
Found 54 images belonging to 3 classes.


# Xception Model

In [2]:
from keras.applications.xception import Xception

xception_base = Xception(input_shape=(224,224,3), include_top=False, weights='imagenet')
xception_base.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [3]:
# Freeze all layers
for layer in xception_base.layers:
    layer.trainable = False
# Unfreeze last 6 layers
for layer in xception_base.layers[-6:]:
    layer.trainable = True

# Check the trainable status of the individual layers
for layer in xception_base.layers:
    if layer.trainable == True:
        print(layer, layer.trainable)

<keras.layers.convolutional.SeparableConv2D object at 0xb310c32e8> True
<keras.layers.normalization.BatchNormalization object at 0xb3110edd8> True
<keras.layers.core.Activation object at 0xb3115e6d8> True
<keras.layers.convolutional.SeparableConv2D object at 0xb31142c88> True
<keras.layers.normalization.BatchNormalization object at 0xb3117df60> True
<keras.layers.core.Activation object at 0xb31258a90> True


In [4]:
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras import optimizers, models

model = models.Sequential()
model.add(xception_base)

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

sgd_opt = optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=True)
Nadam_opt = optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999,
                             epsilon=None, schedule_decay=0.004) # recommended to leave the default params

model.compile(loss='categorical_crossentropy',
              optimizer=Nadam_opt,
              metrics=['accuracy'])

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 7, 7, 2048)        20861480  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               25690368  
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 771       
Total params: 46,552,619
Trainable params: 30,439,939
Non-trainable params: 16,112,680
_________________________________________________________________


In [ ]:
model.fit_generator(generator=trainDataGenerator,
                    steps_per_epoch=trainDataGenerator.samples/trainDataGenerator.batch_size,
                    epochs=10,
                    validation_data=validationDataGenerator,
                    validation_steps=validationDataGenerator.samples/validationDataGenerator.batch_size,
                    verbose=1
                   )

In [ ]:
# Possible hyperparameters:
# Num. of layers to freeze
# Num. of hidden layers in fc layer
# Num. of neurons per hidden layer
# Optimizer learning rate
# Dropout percentage

# Hyperas Testing

In [14]:
def create_model(train_generator, validation_generator):
    model = models.Sequential()
    model.add(Conv2D({{choice([4,8,16])}}, (3,3), input_shape=(224,224,3), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(16, (3,3), input_shape=(224,224,3), activation='relu'))
    model.add(MaxPooling2D())

    model.add(Flatten())
    model.add(Dense(24, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
              optimizer='nadam',
              metrics=['accuracy'])
    
    model.fit_generator(generator=train_generator,
                        steps_per_epoch=train_generator.samples/train_generator.batch_size,
                        epochs=2,
                        validation_data=validation_generator,
                        validation_steps=validation_generator.samples/validation_generator.batch_size,
                        verbose=0
                       )
    
    score, acc = model.evaluate_generator(validation_generator,
                                          steps=validation_generator.samples/validation_generator.batch_size
                                         )
    
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [15]:
def data():
    # Create image augmenter
    dataAugment = ImageDataGenerator(rescale=1.0/255,
                                     rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     zoom_range=0.2,
                                     shear_range=0.2,
                                     fill_mode='reflect',
                                     horizontal_flip=True,
                                     validation_split=0.2,
                                     data_format='channels_last'
                                    )

    train_generator = dataAugment.flow_from_directory(directory='Data/Train',
                                                      target_size=(224, 224), # change this to the input size used for your model
                                                      class_mode='categorical',
                                                      subset='training',
                                                      shuffle=True
                                                      # batch_size=
                                                     )

    validation_generator = dataAugment.flow_from_directory(directory='Data/Train',
                                                           target_size=(224,224), # change this to the input size used for your model
                                                           class_mode='categorical',
                                                           subset='validation',
                                                           shuffle=True
                                                           # batch_size=
                                                          )
    
    
    return train_generator, validation_generator

In [19]:
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras import optimizers, models
from keras.preprocessing.image import ImageDataGenerator

from hyperas import optim
from hyperopt import Trials, STATUS_OK, tpe
from hyperas.distributions import choice, uniform, conditional
import h5py

train_generator, validation_generator = data()

best_run, best_model = optim.minimize(model=create_model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=5,
                                      trials=Trials(),
                                      notebook_name='XCeptionTesting_Jacob'
                                     )

Found 221 images belonging to 3 classes.
Found 54 images belonging to 3 classes.
>>> Imports:
#coding=utf-8

try:
    from keras.preprocessing.image import ImageDataGenerator
except:
    pass

try:
    from keras.applications.xception import Xception
except:
    pass

try:
    from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
except:
    pass

try:
    from keras import optimizers, models
except:
    pass

try:
    from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
except:
    pass

try:
    from keras import optimizers, models
except:
    pass

try:
    from keras.preprocessing.image import ImageDataGenerator
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas.distributions import choice, uniform, conditional
except:
    pass

try:
    import h5py
except:
    pass

>>> Hyperas search space:

def get_space():
  

/Users/jbeallor/anaconda3/envs/test_env/lib/python3.6/site-packages/PIL/Image.py:931: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


7/6 [==============================] - 10s 1s/step - loss: 1.7744 - acc: 0.4061 - val_loss: 1.1284 - val_acc: 0.3519
Epoch 2/2
7/6 [==============================] - 9s 1s/step - loss: 1.0632 - acc: 0.4236 - val_loss: 0.9676 - val_acc: 0.7037
Epoch 1/2
7/6 [==============================] - 8s 1s/step - loss: 4.0114 - acc: 0.3260 - val_loss: 1.0958 - val_acc: 0.4074
Epoch 2/2
7/6 [==============================] - 7s 996ms/step - loss: 1.0895 - acc: 0.3565 - val_loss: 1.0892 - val_acc: 0.4444
Epoch 1/2
7/6 [==============================] - 9s 1s/step - loss: 1.3055 - acc: 0.3886 - val_loss: 1.0399 - val_acc: 0.3889
Epoch 2/2
7/6 [==============================] - 8s 1s/step - loss: 1.0103 - acc: 0.4492 - val_loss: 0.8130 - val_acc: 0.7778
Epoch 1/2
7/6 [==============================] - 10s 1s/step - loss: 2.1348 - acc: 0.3663 - val_loss: 1.0932 - val_acc: 0.5185
Epoch 2/2
7/6 [==============================] - 8s 1s/step - loss: 1.0809 - acc: 0.4093 - val_loss: 1.0501 - val_acc: 0.53

In [20]:
best_run

{'Conv2D': 2}

In [22]:
best_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 109, 109, 16)      2320      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 54, 54, 16)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 46656)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 24)                1119768   
_________________________________________________________________
dropout_5 (Dropout)          (None, 24)                0         
__________